# Download Data from Mockaroo to Dataframes


In [1]:
import pandas as pd

In [2]:
# From google drive - will need access to Community & Growth Shared Google Drive
from google.colab import drive
drive.mount('/content/drive')

marketing_csv_url='/content/drive/Shareddrives/Community & Growth/Needful Things/marketing_spend.csv'
orders_csv_url='/content/drive/Shareddrives/Community & Growth/Needful Things/orders.csv'
reviews_csv_url='/content/drive/Shareddrives/Community & Growth/Needful Things/reviews.csv'
deliveries_csv_url='/content/drive/Shareddrives/Community & Growth/Needful Things/deliveries.csv'
partners_csv_url='/content/drive/Shareddrives/Community & Growth/Needful Things/partners.csv'

Mounted at /content/drive


In [ ]:
# OR From the API (slow!) - and can only get first 5k records - DONT DO THIS
marketing_csv_url='https://api.mockaroo.com/api/2af57370?count=144&key=20f723a0'
orders_csv_url='https://api.mockaroo.com/api/965ce260?count=5000&key=20f723a0'
reviews_csv_url='https://api.mockaroo.com/api/98f38d20?count=5000&key=20f723a0'
deliveries_csv_url='https://api.mockaroo.com/api/0da80a30?count=5000&key=20f723a0'

In [2]:
# Or local
marketing_csv_url='./csv/marketing_spend.csv'
# orders_csv_url='./csv/orders.csv'
orders_csv_url='./csv/orders-small.csv'
reviews_csv_url='./csv/reviews.csv'
deliveries_csv_url='./csv/deliveries.csv'
partners_csv_url='./csv/partners.csv'

In [ ]:
marketing_spend = pd.read_csv(marketing_csv_url)
marketing_spend['month_begin'] = marketing_spend['month_begin'].astype('datetime64[ns]')

In [5]:
orders=pd.read_csv(orders_csv_url)
# convert the 'Date' column to datetime format
orders['order_month'] = orders['order_month'].astype('datetime64[ns]')
orders['order_datetime'] = orders['order_datetime'].astype('datetime64')

In [6]:
orders.dtypes

id                         int64
order_datetime    datetime64[ns]
order_month       datetime64[ns]
first_name                object
last_name                 object
email                     object
address                   object
state                     object
zipcode                    int64
item                      object
category                  object
sales                    float64
channel                   object
channel_group             object
channel_month             object
dtype: object

In [ ]:
reviews=pd.read_csv(reviews_csv_url)
# For reviews it randomly generates a certain number of reviews with random order IDs. Some are duplicates, not desirable. Drop these duplicates so there is never >1 review per order.
reviews.drop_duplicates(subset='order_id',inplace=True)

In [ ]:
reviews.to_csv('reviews_deduped.csv',index=False)

In [ ]:
deliveries=pd.read_csv(deliveries_csv_url)
deliveries['order_datetime'] = deliveries['order_datetime'].astype('datetime64[ns]')
deliveries['delivery_slot_start'] = deliveries['delivery_slot_start'].astype('datetime64[ns]')
deliveries['delivery_slot_end'] = deliveries['delivery_slot_end'].astype('datetime64[ns]')
deliveries['delivery_time'] = deliveries['delivery_time'].astype('datetime64[ns]')


In [ ]:
partners=pd.read_csv(partners_csv_url)
partners['order_datetime'] = partners['order_datetime'].astype('datetime64[ns]')

# (Option 1) Connect to Snowflake and upload db

In [ ]:
!pip install "snowflake-connector-python[pandas]"
!pip install --upgrade sqlalchemy
!pip install snowflake-sqlalchemy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 21.3 MB 1.5 MB/s 
     |████████████████████████████████| 105 kB 50.2 MB/s 
     |████████████████████████████████| 2.3 MB 44.3 MB/s 
     |████████████████████████████████| 194 kB 59.0 MB/s 
     |████████████████████████████████| 57 kB 5.6 MB/s 
     |████████████████████████████████| 29.4 MB 1.2 MB/s 
ERROR: Operation cancelled by user
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 6.7 MB/s 
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.4.44
    Uninstalling SQLAlchemy-1.4.44:
      Successfully uninstalled SQLAlchemy-1.4.44
ERROR: Operation cancelled by user
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Operation cancelled by user


In [ ]:
import snowflake.connector
from sqlalchemy import create_engine
from snowflake.connector.pandas_tools import pd_writer
from snowflake.sqlalchemy import URL


ModuleNotFoundError: ignored

In [ ]:
snowflake_user='devadmin'
snowflake_account='vb69737.us-central1.gcp'
database='NEEDFUL_THINGS'
#database='BLOG'
schema='PUBLIC'
warehouse='COMPUTE_WH'

In [ ]:
# Credentials in 1Password
from getpass import getpass
snowflake_password = getpass('Enter the snowflake password (stored in 1password): ')

Enter the snowflake password (stored in 1password): ··········


In [ ]:
partners

In [ ]:
# Snowflake pandas connector is fussy
marketing_spend.columns=map(str.upper, marketing_spend.columns)
orders.columns=map(str.upper, orders.columns)
reviews.columns=map(str.upper, reviews.columns)
deliveries.columns=map(str.upper, deliveries.columns)
partners.columns=map(str.upper, partners.columns)

# Type inference needs dt.datetime not pandas datetime objects
marketing_spend.MONTH_BEGIN=pd.to_datetime(marketing_spend.MONTH_BEGIN).dt.date

orders.ORDER_DATETIME=pd.to_datetime(orders.ORDER_DATETIME).dt.tz_localize('UTC')
orders.ORDER_MONTH=pd.to_datetime(orders.ORDER_MONTH).dt.date

deliveries.ORDER_DATETIME=pd.to_datetime(deliveries.ORDER_DATETIME).dt.tz_localize('UTC')
deliveries.DELIVERY_SLOT_START=pd.to_datetime(deliveries.DELIVERY_SLOT_START).dt.tz_localize('UTC')
deliveries.DELIVERY_SLOT_END=pd.to_datetime(deliveries.DELIVERY_SLOT_END).dt.tz_localize('UTC')
deliveries.DELIVERY_TIME=pd.to_datetime(deliveries.DELIVERY_TIME).dt.tz_localize('UTC')

partners.ORDER_DATETIME=pd.to_datetime(orders.ORDER_DATETIME).dt.tz_convert('UTC')

In [ ]:
partners.dtypes

ID                              int64
PARTNER                        object
ORDER_DATETIME    datetime64[ns, UTC]
ITEM                           object
CATEGORY                       object
UNIT_PRICE                    float64
QUANTITY                        int64
SALES                         float64
dtype: object

In [ ]:
orders.dtypes

ID                              int64
ORDER_DATETIME    datetime64[ns, UTC]
ORDER_MONTH                    object
FIRST_NAME                     object
LAST_NAME                      object
EMAIL                          object
ADDRESS                        object
STATE                          object
ZIPCODE                         int64
ITEM                           object
CATEGORY                       object
SALES                         float64
CHANNEL                        object
CHANNEL_GROUP                  object
CHANNEL_MONTH                  object
dtype: object

In [ ]:
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine

engine = create_engine(URL(
    account = snowflake_account,
    user=snowflake_user,
    password=snowflake_password,
))

connection = engine.connect()
try:
    connection.execute('''USE DATABASE NEEDFUL_THINGS''')
    #connection.execute('''USE DATABASE BLOG''')
    connection.execute('''USE WAREHOUSE COMPUTE_WH''')


    #connection.execute("ALTER SESSION SET TIMESTAMP_TYPE_MAPPING = 'TIMESTAMP_TZ';")
    #connection.execute("ALTER SESSION SET TIMEZONE = 'UTC';")

    marketing_spend.to_sql('marketing_spend', connection, if_exists='replace', index = False, method = pd_writer)
    orders.to_sql('orders', connection, if_exists='replace', index = False, method = pd_writer)
    reviews.to_sql('reviews', connection, if_exists='replace', index = False, method = pd_writer)
    deliveries.to_sql('deliveries', connection, if_exists='replace', index = False, method = pd_writer)
    partners.to_sql('partners', connection, if_exists='replace', index = False, method = pd_writer)
finally:
    connection.close()
    engine.dispose()

# (Option 2) Put data into SQLite db and save in root folder

In [ ]:
import sqlite3
import pandas as pd

In [ ]:
conn = sqlite3.connect('needful_things.db')
c = conn.cursor()

In [ ]:
# delete any current data  otherwise causes create table commands to fail
# will fail if no tables - this is fine
c.execute(''' DROP TABLE marketing_spend''')
c.execute(''' DROP TABLE reviews''')
c.execute(''' DROP TABLE orders''')
c.execute(''' DROP TABLE deliveries''')

In [ ]:
# Create tables for SQLite db. If you add new columns in mockaroo you will need to add these here

c.execute('''CREATE TABLE marketing_spend (
    id int, 
    channel text, 
    month_begin text, 
    spend double
    )''')

c.execute(
    '''CREATE TABLE reviews (
    id int, 
    order_id int, 
    nps_score int
    )''')

c.execute(
    '''CREATE TABLE orders (
    id int, 
    datetime text, 
    first_name text, 
    last_name text, 
    email text,
    address text,
    state text,
    zipcode text,
    item text,
    category text,
    sales numeric,
    channel text,
    channel_group text
    )''')

c.execute(
    '''CREATE TABLE deliveries (
    id int, 
    order_id int,
    order_datetime text,
    delivery_slot_start text,
    delivery_slot_end text,
    delivery_time text
    )''')

In [ ]:
# Write data to SQLite db
marketing_spend.to_sql('marketing_spend', conn, if_exists='replace', index = False)
orders.to_sql('orders', conn, if_exists='replace', index = False)
reviews.to_sql('reviews', conn, if_exists='replace', index = False)
deliveries.to_sql('deliveries', conn, if_exists='replace', index = False)

# (Option 3) Save to DuckDB

In [5]:
!pip install duckdb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import duckdb

In [46]:
con = duckdb.connect(database='needful_things.duckdb', read_only=False)

In [40]:
con.execute('CREATE TABLE orders AS SELECT * FROM orders')
con.execute('CREATE TABLE deliveries AS SELECT * FROM deliveries')
con.execute('CREATE TABLE marketing_spend AS SELECT * FROM marketing_spend')
con.execute('CREATE TABLE reviews AS SELECT * FROM reviews')
con.execute('CREATE TABLE partners AS SELECT * FROM partners')

In [41]:
con.execute('PRAGMA table_info(orders)').df()

,cid,name,type,notnull,dflt_value,pk
0,0,id,BIGINT,False,NaN,False
1,1,order_datetime,TIMESTAMP,False,NaN,False
2,2,order_month,TIMESTAMP,False,NaN,False
3,3,first_name,VARCHAR,False,NaN,False
4,4,last_name,VARCHAR,False,NaN,False
5,5,email,VARCHAR,False,NaN,False
6,6,address,VARCHAR,False,NaN,False
7,7,state,VARCHAR,False,NaN,False
8,8,zipcode,BIGINT,False,NaN,False
9,9,item,VARCHAR,False,NaN,False


In [50]:
con.execute("copy orders to 'orders.parquet'")

In [45]:
con.close()

# (Option 4) Save to Supabase

In [ ]:
!pip install supabase

In [ ]:
# Credentials in 1Password
from getpass import getpass
url = getpass('Enter the supabase url (stored in 1password): ')
key = getpass('Enter the supabase key (stored in 1password): ')

Enter the supabase url (stored in 1password): ··········
Enter the supabase key (stored in 1password): ··········


In [ ]:
from supabase import create_client, Client

supabase: Client = create_client(url, key)
data = supabase.table("orders").insert({"name":"orders"}).execute()
assert len(data.data) > 0

# Data Checks (not req'd)


In [ ]:
orders

,id,order_datetime,order_month,first_name,last_name,email,address,state,zipcode,item,category,sales,channel,channel_group,channel_month
0,1,2020-07-04 16:39:03,2020-07-01,Shannon,Tregenna,stregenna0@tripadvisor.com,7 Porter Lane,California,91109,Model Racehorse,Sinister Toys,13.00,Referral,Referral,Referral2020-07-01
1,2,2021-08-09 00:48:50,2021-08-01,Myrwyn,Sibbson,msibbson1@chronoengine.com,661 Vera Trail,California,94064,Toy Doll,Sinister Toys,5.00,Google Organic,Organic Search,Google Organic2021-08-01
2,3,2019-02-04 04:41:54,2019-02-01,Tansy,Carlesso,tcarlesso2@miitbeian.gov.cn,8 Warner Place,Nevada,89166,Necklace,Mysterious Apparel,13.00,Facebook Ads,Social,Facebook Ads2019-02-01
3,4,2019-04-05 05:12:34,2019-04-01,Dale,Melloy,dmelloy3@sun.com,23 Hagan Point,Oregon,97240,Lamp,Odd Equipment,34.00,Tiktok Ads,Social,Tiktok Ads2019-04-01
4,5,2021-09-01 20:47:34,2021-09-01,Ezra,Belli,ebelli4@squarespace.com,24 Hooker Way,Texas,78726,Baseball Card,Sinister Toys,3.00,Google Paid,Paid Search,Google Paid2021-09-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,2996,2020-02-10 02:20:00,2020-02-01,Ethan,Donlon,edonlon2b7@census.gov,11 Acker Terrace,Oklahoma,73114,Fishing Rod,Cursed Sporting Goods,89.00,Google Organic,Organic Search,Google Organic2020-02-01
2996,2997,2019-10-07 14:55:26,2019-10-01,Taddeo,Isted,tisted2b8@amazonaws.com,880 Almo Parkway,Florida,32220,Baseball Card,Sinister Toys,2.85,Google Organic,Organic Search,Google Organic2019-10-01
2997,2998,2020-06-13 23:48:29,2020-06-01,Arlyne,Berger,aberger2b9@rediff.com,788 7th Park,Texas,78225,Lamp,Odd Equipment,34.00,Facebook Ads,Social,Facebook Ads2020-06-01
2998,2999,2021-03-29 23:36:30,2021-03-01,Meagan,Toe,mtoe2ba@nyu.edu,34 Del Mar Trail,Missouri,64054,Fishing Rod,Cursed Sporting Goods,89.00,Google Paid,Paid Search,Google Paid2021-03-01


In [ ]:
pd.read_sql('select * from orders limit 3000', conn)

,id,order_datetime,order_month,first_name,last_name,email,address,state,zipcode,item,category,sales,channel,channel_group,channel_month
0,1,2021-04-28 07:55:50,2021-04-01,Frederic,Van der Velden,fvandervelden0@cargocollective.com,47009 Derek Hill,Kansas,66629,Model Racehorse,Sinister Toys,13.00,Google Organic,Organic Search,Google Organic2021-04-01
1,2,2021-05-14 09:45:12,2021-05-01,Evelin,Staig,estaig1@storify.com,71 Monterey Park,California,92812,Lamp,Odd Equipment,34.00,Google Paid,Paid Search,Google Paid2021-05-01
2,3,2020-01-09 04:54:08,2020-01-01,Bebe,Kilgrove,bkilgrove2@icio.us,759 Vidon Alley,Texas,78759,Model Racehorse,Sinister Toys,13.00,Facebook Ads,Social,Facebook Ads2020-01-01
3,4,2021-02-28 10:19:54,2021-02-01,Cristine,Ardy,cardy3@examiner.com,6487 Lakewood Gardens Junction,California,91131,Lamp,Odd Equipment,34.00,Referral,Referral,Referral2021-02-01
4,5,2021-10-10 21:16:55,2021-10-01,Rosella,Cosgry,rcosgry4@ox.ac.uk,13 Springview Court,Florida,33680,Lamp,Odd Equipment,34.00,Facebook Ads,Social,Facebook Ads2021-10-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,2996,2021-01-23 11:46:14,2021-01-01,Forrest,Cail,fcail2b7@chron.com,79745 Jay Alley,Pennsylvania,19160,Baseball Card,Sinister Toys,3.00,Google Organic,Organic Search,Google Organic2021-01-01
2996,2997,2021-11-18 10:14:20,2021-11-01,Forester,Escala,fescala2b8@aol.com,35656 4th Lane,Washington,98424,Typewriter,Odd Equipment,89.00,Google Organic,Organic Search,Google Organic2021-11-01
2997,2998,2021-07-21 15:14:01,2021-07-01,Isiahi,Darbishire,idarbishire2b9@squidoo.com,275 Emmet Avenue,New York,10474,Lamp,Odd Equipment,34.00,Google Organic,Organic Search,Google Organic2021-07-01
2998,2999,2020-04-01 05:51:48,2020-04-01,Mariam,Dursley,mdursley2ba@weebly.com,0 Stuart Crossing,Arizona,85246,Running Shoes,Cursed Sporting Goods,55.00,Facebook Ads,Social,Facebook Ads2020-04-01


In [ ]:
orders=pd.read_csv(orders_csv_url)

In [ ]:
import plotly.express as px

In [ ]:
orders['date'] = pd.to_datetime(orders['order_datetime']).dt.date
orders['year'] = pd.to_datetime(orders['order_datetime']).dt.year
orders['month'] = orders['order_datetime'].astype('datetime64[M]')

In [ ]:
orders

,id,order_datetime,order_month,first_name,last_name,email,address,state,zipcode,item,category,sales,channel,channel_group,channel_month,date,year,month
0,1,2019-03-18 18:03:17,2019-03-01,Devlen,Coy,dcoy0@hao123.com,91 Kensington Center,Wisconsin,53405,Necklace,Mysterious Apparel,12.35,Google Organic,Organic Search,Google Organic2019-03-01,2019-03-18,2019,2019-03-01
1,2,2020-01-28 16:21:46,2020-01-01,Jocko,Addinall,jaddinall1@ycombinator.com,16950 Spohn Lane,New York,12210,Toy Doll,Sinister Toys,5.00,Coupon,Other,Coupon2020-01-01,2020-01-28,2020,2020-01-01
2,3,2020-01-01 11:04:02,2020-01-01,Erroll,O'Collopy,eocollopy2@omniture.com,262 Granby Trail,South Carolina,29403,Baseball Card,Sinister Toys,2.70,Google Paid,Paid Search,Google Paid2020-01-01,2020-01-01,2020,2020-01-01
3,4,2019-09-10 02:48:24,2019-09-01,Nyssa,Vallow,nvallow3@bravesites.com,176 Basil Park,Georgia,30033,Model Racehorse,Sinister Toys,13.00,Facebook Ads,Social,Facebook Ads2019-09-01,2019-09-10,2019,2019-09-01
4,5,2020-05-15 21:33:59,2020-05-01,Curtice,Gudgion,cgudgion4@netvibes.com,838 Beilfuss Alley,Georgia,31704,Vintage Jacket,Mysterious Apparel,34.00,Google Organic,Organic Search,Google Organic2020-05-01,2020-05-15,2020,2020-05-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,2996,2019-01-27 13:41:20,2019-01-01,Mehetabel,Chandler,mchandler2b7@quantcast.com,6 Dorton Avenue,Colorado,81015,Vintage Jacket,Mysterious Apparel,32.30,Google Paid,Paid Search,Google Paid2019-01-01,2019-01-27,2019,2019-01-01
2996,2997,2019-07-23 09:30:34,2019-07-01,Judi,Mateja,jmateja2b8@vk.com,775 Ilene Road,Hawaii,96805,Boxing Gloves,Cursed Sporting Goods,21.00,Facebook Ads,Social,Facebook Ads2019-07-01,2019-07-23,2019,2019-07-01
2997,2998,2020-08-01 02:05:28,2020-08-01,Bertina,Warnes,bwarnes2b9@sohu.com,7938 Northridge Way,New Jersey,8104,Running Shoes,Cursed Sporting Goods,55.00,Google Paid,Paid Search,Google Paid2020-08-01,2020-08-01,2020,2020-08-01
2998,2999,2019-02-13 13:47:08,2019-02-01,Emlen,Pinckstone,epinckstone2ba@feedburner.com,8889 Rutledge Trail,Ohio,45271,Lamp,Odd Equipment,32.30,Google Organic,Organic Search,Google Organic2019-02-01,2019-02-13,2019,2019-02-01


In [ ]:
orders_per_day=orders.pivot_table(values='sales', index = 'date')
orders_per_month=orders.pivot_table(values='sales', index = 'month', aggfunc=sum)
orders_per_year=orders.pivot_table(values='sales', index = 'year', aggfunc=sum)

In [ ]:
px.bar(orders_per_year, y='sales')

In [ ]:
marketing_spend

,id,channel,month_begin,spend,channel_month
0,1,Google Paid,2019-01-01,525.54,Google Paid2019-01-01
1,2,Google Paid,2019-02-01,560.15,Google Paid2019-02-01
2,3,Google Paid,2019-03-01,567.67,Google Paid2019-03-01
3,4,Google Paid,2019-04-01,860.00,Google Paid2019-04-01
4,5,Google Paid,2019-05-01,921.74,Google Paid2019-05-01
...,...,...,...,...,...
139,140,Tiktok Ads,2021-08-01,1200.07,Tiktok Ads2021-08-01
140,141,Tiktok Ads,2021-09-01,1196.00,Tiktok Ads2021-09-01
141,142,Tiktok Ads,2021-10-01,1175.74,Tiktok Ads2021-10-01
142,143,Tiktok Ads,2021-11-01,1564.63,Tiktok Ads2021-11-01


In [ ]:
marketing_spend.join(orders,how='left',on='channel_month')

In [ ]:
reviews

,id,order_id,nps_score
0,1,2379,9
1,2,1961,10
2,3,2733,6
3,4,2316,8
4,5,1309,7
...,...,...,...
494,495,2437,8
495,496,2738,2
496,497,200,7
498,499,2111,10


In [ ]:
px.histogram(reviews, x='nps_score', width= 600, height = 400)

In [ ]:
px.histogram(reviews, x='nps_score')